In [17]:
import pandas as pd
import seaborn as sns
import networkx as nx
import numpy as np
import re
import matplotlib.pyplot as plt
import itertools
import sklearn.datasets
from sklearn.metrics.pairwise import euclidean_distances
import math

In [18]:
data = sklearn.datasets.load_iris()
df = pd.DataFrame(data=data.data, columns=data.feature_names)

In [19]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [20]:
dfNumeric = df.select_dtypes(include=np.number)
dfNumeric

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [21]:
# Similarity matrix -> Has weird values, bit different, perhaps I need to make my own distance matrix
distances = euclidean_distances(df.values)
sigma = 1
distances = distances / -(2 * pow(sigma, 2))
distances = np.exp(distances)

In [22]:
def kNNAlgorithm(k, distances):
    edgeList = {}
    for i in range(0, len(distances)):
        edgeList[i] = []
        for n in range(0, k):
            shortestDistance = -1
            shortestIndex = -1
            for j in range(0, len(distances)):
                if distances[i][j] >= shortestDistance and i != j: # I called it distance but its actually similarity, the higher the better
                    shortestDistance = distances[i][j]
                    shortestIndex = j
            distances[i][shortestIndex] = -1 # Set to inf distance thats already found to be closest
            edgeList[i].append((shortestIndex, shortestDistance))
    return edgeList

In [23]:
kNNResult = kNNAlgorithm(3, distances=np.copy(distances))
kNNResult

{0: [(17, 0.9512294245007188),
  (28, 0.9317314234234011),
  (4, 0.9317314234234011)],
 1: [(34, 0.9317314234233893),
  (12, 0.9317314234233893),
  (45, 0.9317314234233777)],
 2: [(47, 0.9317314234233893),
  (3, 0.8847284766100587),
  (12, 0.8760890258268804)],
 3: [(47, 0.9317314234234011),
  (29, 0.9170415101798975),
  (30, 0.8942200448866195)],
 4: [(37, 0.9317314234234011),
  (0, 0.9317314234234011),
  (40, 0.9170415101799163)],
 5: [(18, 0.8471891941830079),
  (10, 0.840965131393053),
  (48, 0.8350384028320621)],
 6: [(47, 0.8942200448866336),
  (2, 0.8760890258268745),
  (11, 0.8607079764250554)],
 7: [(39, 0.9512294245007357),
  (49, 0.9317314234234011),
  (0, 0.9170415101799163)],
 8: [(38, 0.9317314234233893),
  (3, 0.8607079764250554),
  (42, 0.8537525485232912)],
 9: [(34, 0.9512294245007188),
  (30, 0.9170415101799069),
  (1, 0.9170415101799069)],
 10: [(48, 0.9512294245007188),
  (27, 0.8681234453945862),
  (36, 0.8607079764250656)],
 11: [(7, 0.8942200448866267),
  (29, 0

In [ ]:
def networkToDataframe(edgeList : dict):
    result = []
    i = 0
    for key, value in edgeList.items():
        for pair in value:
            result.append([])
            result[i].append(key)
            result[i].append(pair[0])
            result[i].append(pair[1])
            i += 1
    return result


In [25]:
def eRadius(epsilon, distances):
    edgeList = {}
    for i in range(0, len(distances)):
        edgeList[i] = []
    for i in range(0, len(distances)):
        for j in range(0, len(distances)):
            if distances[i][j] > epsilon and j not in edgeList[i] and i != j: # If similarity is higher than 0.9 (epsilon)? 
                edgeList[i].append((j, distances[i][j]))
    return edgeList

In [26]:
dfKNN = pd.DataFrame(data=networkToDataframe(edgeList=kNNResult), columns=['from', 'to', 'similarity'])

In [33]:
dfKNN.to_csv("KNNAlgorithmResult.csv")
display(dfKNN)

,from,to,similarity
0,0,17,0.951229
1,0,28,0.931731
2,0,4,0.931731
3,1,34,0.931731
4,1,12,0.931731
...,...,...,...
445,148,115,0.860708
446,148,110,0.757003
447,149,127,0.868123
448,149,138,0.853753


In [36]:
resultERadius = eRadius(0.9, np.copy(distances))
print(resultERadius)
dfERadius = pd.DataFrame(data=networkToDataframe(edgeList=resultERadius), columns=['from', 'to', 'similarity'])
display(dfERadius)
dfERadius.to_csv("eRadiusResult.csv")

{0: [(4, 0.9317314234234011), (7, 0.9170415101799163), (17, 0.9512294245007188), (27, 0.9317314234233893), (28, 0.9317314234234011), (39, 0.9317314234233893), (40, 0.9170415101799163)], 1: [(9, 0.9170415101799069), (12, 0.9317314234233893), (34, 0.9317314234233893), (45, 0.9317314234233777)], 2: [(47, 0.9317314234233893)], 3: [(29, 0.9170415101798975), (47, 0.9317314234234011)], 4: [(0, 0.9317314234234011), (17, 0.9170415101799163), (37, 0.9317314234234011), (40, 0.9170415101799163)], 5: [], 6: [], 7: [(0, 0.9170415101799163), (17, 0.9048374180359686), (39, 0.9512294245007357), (49, 0.9317314234234011)], 8: [(38, 0.9317314234233893)], 9: [(1, 0.9170415101799069), (12, 0.9170415101798975), (25, 0.9048374180359444), (30, 0.9170415101799069), (34, 0.9512294245007188)], 10: [(48, 0.9512294245007188)], 11: [], 12: [(1, 0.9317314234233893), (9, 0.9170415101798975), (34, 0.9048374180359686), (45, 0.9048374180359605)], 13: [], 14: [], 15: [], 16: [], 17: [(0, 0.9512294245007188), (4, 0.9170415

,from,to,similarity
0,0,4,0.931731
1,0,7,0.917042
2,0,17,0.951229
3,0,27,0.931731
4,0,28,0.931731
...,...,...,...
125,132,128,0.951229
126,137,116,0.931731
127,138,127,0.931731
128,139,112,0.917042


In [29]:
def kNNERadius(k, epsilon, distances):
    edgeList = {}
    for i in range(0, len(distances)):
        edgeList[i] = []
        for n in range(0, k):
            shortestDistance = -1
            shortestIndex = -1
            for j in range(0, len(distances)):
                if distances[i][j] >= shortestDistance and i != j and distances[i][j] > epsilon: # I called it distance but its actually similarity, the higher the better
                    shortestDistance = distances[i][j]
                    shortestIndex = j
            distances[i][shortestIndex] = -1 # Set to inf distance thats already found to be closest
            if shortestIndex != -1:
                edgeList[i].append((shortestIndex, shortestDistance))
    return edgeList

In [30]:
resultCombi = kNNERadius(3, 0.9, np.copy(distances))
resultCombi

{0: [(17, 0.9512294245007188),
  (28, 0.9317314234234011),
  (4, 0.9317314234234011)],
 1: [(34, 0.9317314234233893),
  (12, 0.9317314234233893),
  (45, 0.9317314234233777)],
 2: [(47, 0.9317314234233893)],
 3: [(47, 0.9317314234234011), (29, 0.9170415101798975)],
 4: [(37, 0.9317314234234011),
  (0, 0.9317314234234011),
  (40, 0.9170415101799163)],
 5: [],
 6: [],
 7: [(39, 0.9512294245007357),
  (49, 0.9317314234234011),
  (0, 0.9170415101799163)],
 8: [(38, 0.9317314234233893)],
 9: [(34, 0.9512294245007188),
  (30, 0.9170415101799069),
  (1, 0.9170415101799069)],
 10: [(48, 0.9512294245007188)],
 11: [],
 12: [(1, 0.9317314234233893),
  (9, 0.9170415101798975),
  (34, 0.9048374180359686)],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [(0, 0.9512294245007188),
  (40, 0.9317314234234011),
  (4, 0.9170415101799163)],
 18: [],
 19: [(46, 0.9317314234233893), (21, 0.9317314234233893)],
 20: [],
 21: [(19, 0.9317314234233893)],
 22: [],
 23: [(26, 0.9048374180359605)],
 24: [],
 25: [(34, 0.

In [37]:
dfCombi = pd.DataFrame(data=networkToDataframe(edgeList=resultCombi), columns=['from', 'to', 'similarity'])
display(dfCombi)
dfCombi.to_csv("CombinationResult.csv")

,from,to,similarity
0,0,17,0.951229
1,0,28,0.931731
2,0,4,0.931731
3,1,34,0.931731
4,1,12,0.931731
...,...,...,...
103,132,128,0.951229
104,137,116,0.931731
105,138,127,0.931731
106,139,112,0.917042


In [31]:
distances

array([[1.        , 0.76394595, 0.77495449, ..., 0.10753807, 0.09774403,
        0.12618273],
       [0.76394595, 1.        , 0.86070798, ..., 0.1054578 , 0.09451231,
        0.12534868],
       [0.77495449, 0.86070798, 1.        , ..., 0.09722062, 0.08853515,
        0.11655191],
       ...,
       [0.10753807, 0.1054578 , 0.09722062, ..., 1.        , 0.73475314,
        0.72603561],
       [0.09774403, 0.09451231, 0.08853515, ..., 0.73475314, 1.        ,
        0.68109241],
       [0.12618273, 0.12534868, 0.11655191, ..., 0.72603561, 0.68109241,
        1.        ]])

In [32]:
# TODO: Generate csv for 2 more methods, check distance values, might be wrong